In [2]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
stop = set(stopwords.words('english'))
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import string
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
import csv
from csv import reader
tit = []
with open("/COVID-19_title.csv", 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        tit.append(row)

In [5]:
tit[:10]

[['clinical features culture-proven mycoplasma pneumoniae infections king abdulaziz university hospital jeddah saudi arabia'],
 ['nitric oxide pro-inflammatory mediator lung diseas'],
 ['surfactant protein-d pulmonary host defens'],
 ['role endothelin-1 lung diseas'],
 ['gene expression epithelial cells response pneumovirus infect'],
 ['sequence requirements strand transfer nidovirus discontinuous subgenomic synthesi'],
 ['debate transfusing normal haemoglobin levels improve outcom'],
 ['21st international symposium intensive care emergency medicine brussels belgium 20-23 march 2001'],
 ['heme oxygenase-1 carbon monoxide pulmonary medicin'],
 ['technical description rods real-time public health surveillance system']]

In [6]:
val = []
for i in tit:
  val.append(i[0])

In [8]:
val[:10]
print(len(val))

119492


Splitting the dataset into 2 equal halves

In [13]:
from sklearn.model_selection import train_test_split
data1,data2 = train_test_split(val, train_size=0.5,random_state=42)
print(data1[:5])
print(data2[:5])

['building international consortium tracking coronavirus health status', 'unparalleled spirit collaboration amid covid-19 pandem', 'insights gained respiratory infection pathogenesis using lung tissue metabolom', 'ubiquitin-proteasome system plays important role various stages coronavirus infection cycl', 'qualitative enquiry strategic operational responses covid-19 challenges south asia']
['covid-19 death italy physician', 'comparison 2-year results roux-en-y gastric bypass transit bipartition sleeve gastrectomy superobes', 'required confronting approaches efficacy time control covid-19 outbreak iran', 'references abbrevi', 'ultrasound assessment gastric contents guidance anaesthetic strategy infants hypertrophic pyloric stenosis prospective cohort studi']


**Running LDA individually on the data1**

In [15]:
#Using bag of words

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = set(stopwords.words('english'))
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
doc_set = data1
# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
corpus[:2]


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)],
 [(7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]]

In [16]:
preview = corpus[-1]
for i in range(len(preview)):
    print("Word {} (\"{}\") appears {} time.".format(preview[i][0], 
                                               dictionary[preview[i][0]], preview[i][1]))

Word 333 ("2") appears 1 time.
Word 370 ("enzyme") appears 1 time.
Word 445 ("cov") appears 1 time.
Word 450 ("sars") appears 1 time.
Word 455 ("infect") appears 1 time.
Word 1193 ("angiotensin") appears 1 time.
Word 1580 ("ace2") appears 1 time.
Word 1581 ("converting") appears 1 time.
Word 2376 ("mitigate") appears 1 time.
Word 2897 ("inhaled") appears 1 time.
Word 3070 ("modified") appears 1 time.
Word 14366 ("decoy") appears 1 time.


In [17]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.4557075019538014),
 (1, 0.5319316202376427),
 (2, 0.17087494430152386),
 (3, 0.20690231636889916),
 (4, 0.3352338220026479),
 (5, 0.3584381790766469),
 (6, 0.44329782912984356)]


In [18]:
lda_model = gensim.models.LdaMulticore(corpus, num_topics=8, id2word=dictionary, passes=2, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.038*"sars" + 0.034*"2" + 0.034*"cov" + 0.016*"coronavirus" + 0.014*"respiratory" + 0.012*"virus" + 0.011*"protein" + 0.009*"viral" + 0.009*"human" + 0.008*"detection"
Topic: 1 
Words: 0.022*"laparoscopic" + 0.018*"single" + 0.015*"surgery" + 0.008*"review" + 0.008*"cancer" + 0.006*"surgical" + 0.006*"assisted" + 0.006*"systematic" + 0.005*"hernia" + 0.005*"repair"
Topic: 2 
Words: 0.017*"health" + 0.014*"review" + 0.013*"acute" + 0.010*"respiratory" + 0.009*"systematic" + 0.008*"care" + 0.007*"severe" + 0.006*"public" + 0.006*"studi" + 0.006*"syndrome"
Topic: 3 
Words: 0.092*"covid" + 0.091*"19" + 0.014*"coronavirus" + 0.013*"pandemic" + 0.013*"patients" + 0.011*"disease" + 0.010*"pandem" + 0.009*"health" + 0.008*"2019" + 0.008*"care"
Topic: 4 
Words: 0.009*"treatment" + 0.008*"risk" + 0.008*"disease" + 0.007*"patients" + 0.005*"case" + 0.005*"approach" + 0.005*"artery" + 0.004*"report" + 0.004*"studi" + 0.004*"aneurysm"
Topic: 5 
Words: 0.010*"acute" + 0.009*"patien

In [19]:
lda_model = gensim.models.LdaMulticore(corpus, num_topics=7, id2word=dictionary, passes=40, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.052*"covid" + 0.051*"19" + 0.020*"health" + 0.014*"review" + 0.012*"pandemic" + 0.009*"disease" + 0.009*"coronavirus" + 0.009*"systematic" + 0.006*"impact" + 0.006*"outbreak"
Topic: 1 
Words: 0.042*"sars" + 0.036*"2" + 0.036*"cov" + 0.030*"respiratory" + 0.026*"coronavirus" + 0.020*"acute" + 0.017*"severe" + 0.015*"syndrome" + 0.013*"infection" + 0.012*"patients"
Topic: 2 
Words: 0.101*"covid" + 0.099*"19" + 0.017*"patients" + 0.015*"pandem" + 0.015*"care" + 0.010*"pandemic" + 0.008*"during" + 0.007*"management" + 0.005*"clinical" + 0.005*"emergency"
Topic: 3 
Words: 0.029*"virus" + 0.013*"protein" + 0.012*"human" + 0.011*"coronavirus" + 0.009*"cell" + 0.008*"infectious" + 0.007*"viral" + 0.006*"vaccine" + 0.006*"expression" + 0.005*"diseas"
Topic: 4 
Words: 0.014*"laparoscopic" + 0.010*"patients" + 0.010*"single" + 0.009*"treatment" + 0.008*"studi" + 0.008*"surgery" + 0.007*"using" + 0.006*"versus" + 0.005*"randomized" + 0.005*"tri"
Topic: 5 
Words: 0.006*"multiple"

In [21]:
pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=bdd939aa2790dcea30f363c9276692795d7eac49a6207d59361e554e144d0bbe
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [23]:
import pyLDAvis
import pyLDAvis.gensim 

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.223121 -0.000720       1        1  21.573287
4     -0.060844  0.082650       2        1  19.082264
1     -0.010913  0.170821       3        1  15.093401
3      0.189382  0.145065       4        1  14.530672
2     -0.246971 -0.078605       5        1  14.516324
6      0.228868 -0.006648       6        1   8.739023
5      0.123598 -0.312562       7        1   6.465028, topic_info=            Term          Freq         Total Category  logprob  loglift
10         covid  14589.000000  14589.000000  Default  30.0000  30.0000
7             19  14344.000000  14344.000000  Default  29.0000  29.0000
450         sars   3568.000000   3568.000000  Default  28.0000  28.0000
333            2   3050.000000   3050.000000  Default  27.0000  27.0000
445          cov   3038.000000   3038.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
124         2020    156.001122    490.833451   Topic7  -5.4525   1.5925
22         using     90.533969   1585.086577   Topic7  -5.9967  -0.1239
930         time     82.885215    909.825867   Topic7  -6.0850   0.3430
261       report     72.366073    902.440781   Topic7  -6.2207   0.2154
3013  scientific     65.445608    138.444220   Topic7  -6.3212   1.9895

[394 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2309      4  0.644186        1
2309      6  0.287878        1
2309      7  0.066070        1
7         1  0.433396       19
7         5  0.566545       19
...     ...       ...      ...
312       2  0.344944     with
312       3  0.300435     with
312       5  0.290698     with
932       1  0.998167     work
2102      1  0.997002  workers

[547 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 2, 4, 3, 7, 6])

In [26]:
lda_model_8_topics = gensim.models.LdaMulticore(corpus, num_topics=8, id2word=dictionary, passes=2, workers=2)

for idx, topic in lda_model_8_topics.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

Topic: 0 
Words: 0.089*"covid" + 0.088*"19" + 0.020*"health" + 0.018*"pandemic" + 0.013*"care" + 0.013*"pandem" + 0.007*"impact" + 0.006*"during" + 0.006*"review" + 0.006*"public"
Topic: 1 
Words: 0.034*"virus" + 0.012*"cell" + 0.011*"protein" + 0.011*"respiratory" + 0.010*"human" + 0.009*"influenza" + 0.007*"viral" + 0.007*"coronavirus" + 0.006*"cells" + 0.005*"vaccine"
Topic: 2 
Words: 0.041*"coronavirus" + 0.032*"sars" + 0.027*"cov" + 0.027*"2" + 0.023*"disease" + 0.018*"respiratory" + 0.017*"covid" + 0.016*"2019" + 0.016*"19" + 0.015*"infection"
Topic: 3 
Words: 0.009*"randomized" + 0.008*"tri" + 0.008*"laparoscopic" + 0.007*"controlled" + 0.007*"term" + 0.006*"surgery" + 0.006*"long" + 0.006*"studi" + 0.005*"patients" + 0.005*"gastric"
Topic: 4 
Words: 0.022*"covid" + 0.022*"19" + 0.007*"using" + 0.007*"based" + 0.007*"model" + 0.006*"data" + 0.006*"epidemic" + 0.006*"analysis" + 0.005*"system" + 0.005*"risk"
Topic: 5 
Words: 0.013*"case" + 0.011*"treatment" + 0.010*"report" + 0.0

In [27]:
import pyLDAvis
import pyLDAvis.gensim 

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_8_topics, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.198767 -0.173684       1        1  20.839915
2      0.115515 -0.123074       2        1  15.076083
1      0.287072  0.008556       3        1  14.515015
7     -0.061003  0.062107       4        1  11.471400
3     -0.068536  0.198042       5        1  10.630440
5     -0.061236  0.138363       6        1  10.304804
4     -0.020997 -0.136441       7        1  10.120644
6      0.007953  0.026131       8        1   7.041699, topic_info=            Term          Freq         Total Category  logprob  loglift
10         covid  14766.000000  14766.000000  Default  30.0000  30.0000
7             19  14528.000000  14528.000000  Default  29.0000  29.0000
2    coronavirus   4138.000000   4138.000000  Default  28.0000  28.0000
450         sars   3539.000000   3539.000000  Default  27.0000  27.0000
143        virus   3105.000000   3105.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
10         covid    190.459139  14766.650085   Topic8  -5.3384  -1.6974
7             19    178.474349  14528.038056   Topic8  -5.4034  -1.7461
90         studi    103.990922   1541.867193   Topic8  -5.9435  -0.0431
235     patients    104.097605   3469.971331   Topic8  -5.9425  -0.8533
321     clinical     96.205232   1984.654958   Topic8  -6.0214  -0.3734

[678 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
5562      7  0.968086      0
2309      2  0.052609      1
2309      3  0.674349      1
2309      4  0.007174      1
2309      5  0.004783      1
...     ...       ...    ...
1718      6  0.051202  wuhan
1938      1  0.067652      y
1938      4  0.840813      y
1938      6  0.057987      y
1938      8  0.019329      y

[2155 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 8, 4, 6, 5, 7])

**Running LDA individually on data2**

In [28]:
#Using bag of words

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = set(stopwords.words('english'))
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
doc_set = data2
# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
corpus[:2]


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1)]]

In [29]:
preview = corpus[-1]
for i in range(len(preview)):
    print("Word {} (\"{}\") appears {} time.".format(preview[i][0], 
                                               dictionary[preview[i][0]], preview[i][1]))

Word 348 ("treatment") appears 1 time.
Word 410 ("cell") appears 2 time.
Word 480 ("risk") appears 1 time.
Word 576 ("factors") appears 1 time.
Word 616 ("outcom") appears 1 time.
Word 783 ("following") appears 1 time.
Word 874 ("transplantation") appears 1 time.
Word 1129 ("disorders") appears 1 time.
Word 1635 ("stem") appears 1 time.
Word 3382 ("hematopoietic") appears 1 time.
Word 30992 ("lymphoproliferative") appears 1 time.


In [30]:
lda_model = gensim.models.LdaMulticore(corpus, num_topics=8, id2word=dictionary, passes=40, workers=3, random_state=42)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

Topic: 0 
Words: 0.074*"covid" + 0.073*"19" + 0.025*"patients" + 0.023*"coronavirus" + 0.021*"review" + 0.019*"disease" + 0.015*"clinical" + 0.015*"2019" + 0.013*"systematic" + 0.009*"care"
Topic: 1 
Words: 0.070*"sars" + 0.059*"cov" + 0.059*"2" + 0.010*"drug" + 0.006*"infection" + 0.006*"coronavirus" + 0.005*"angiotensin" + 0.005*"potential" + 0.004*"molecular" + 0.004*"human"
Topic: 2 
Words: 0.031*"case" + 0.020*"report" + 0.011*"associated" + 0.011*"2020" + 0.008*"united" + 0.007*"risk" + 0.007*"acute" + 0.006*"mortality" + 0.006*"pulmonary" + 0.005*"st"
Topic: 3 
Words: 0.014*"patients" + 0.011*"treatment" + 0.009*"acute" + 0.007*"lung" + 0.007*"studi" + 0.007*"stroke" + 0.007*"therapy" + 0.006*"tri" + 0.005*"artery" + 0.005*"liver"
Topic: 4 
Words: 0.034*"virus" + 0.028*"respiratory" + 0.017*"coronavirus" + 0.014*"acute" + 0.014*"human" + 0.013*"syndrome" + 0.011*"viral" + 0.011*"protein" + 0.011*"severe" + 0.010*"influenza"
Topic: 5 
Words: 0.069*"covid" + 0.067*"19" + 0.022*"he

In [31]:
import pyLDAvis
import pyLDAvis.gensim 

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.245474 -0.091239       1        1  19.381981
0      0.255616 -0.056406       2        1  17.115079
4     -0.137403 -0.208376       3        1  16.062722
3      0.002469  0.192828       4        1  12.712528
7     -0.041902  0.252423       5        1   9.827071
1     -0.175336 -0.157758       6        1   9.274341
2      0.023870  0.012533       7        1   7.935677
6     -0.172789  0.055994       8        1   7.690602, topic_info=         Term          Freq         Total Category  logprob  loglift
1       covid  14597.000000  14597.000000  Default  30.0000  30.0000
0          19  14326.000000  14326.000000  Default  29.0000  29.0000
139      sars   3660.000000   3660.000000  Default  28.0000  28.0000
135       cov   3074.000000   3074.000000  Default  27.0000  27.0000
5           2   3062.000000   3062.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
1845    rapid    136.746541    457.343697   Topic8  -5.7584   1.3579
321     novel    161.196963   1364.989264   Topic8  -5.5939   0.4289
248     model    146.308598   1138.326834   Topic8  -5.6908   0.5136
655   imaging    118.454888    391.801552   Topic8  -5.9020   1.3689
3387     acid    112.369954    324.590624   Topic8  -5.9548   1.5044

[492 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
2556      3  0.838667     1
2556      4  0.010197     1
2556      5  0.025491     1
2556      6  0.086671     1
2556      8  0.038237     1
...     ...       ...   ...
255       8  0.986895     x
17        5  0.991964     y
18        4  0.178202  year
18        5  0.208830  year
18        7  0.612569  year

[807 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 5, 4, 8, 2, 3, 7])